<h1 align="center">Study specific gross pathology visualization</h1> 

__Created by: Rance Nault, Michigan State University__<br>

---
This Jupyter notebook presents a use case for the [toxdatacommons](https://fairtox.com) to plot basic endpoints such as body weight, food consumption, and terminal endpoints.

## Set up your environment and packages

In [22]:
import os
import sys
import csv
import gen3
import json
import uuid
import mwtab
import warnings
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np


#sys.path.append('c://Users/15177/OneDrive - Michigan State University/Documents/Projects/FAIRTox/PythonShared')
from io import StringIO
from datetime import datetime
from collections import OrderedDict
from expansion import Gen3Expansion # https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py

from gen3.submission import Gen3Submission
from gen3.auth import Gen3Auth
from gen3.index import Gen3Index
from gen3.query import Gen3Query
from gen3.metadata import Gen3Metadata
from gen3.file import Gen3File
from TDCexpansion import Gen3process  # https://raw.githubusercontent.com/naultran/toxdatacommons/main/script/TDCexpansion.py

current_date = datetime.now().date()
formatted_date = current_date.strftime('%Y-%m-%d')

warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
api = 'https://fairtox.com/'
cred = 'c://Users/15177/OneDrive - Michigan State University/Documents/Projects/FAIRTox/PythonShared/credentials.json'
auth = Gen3Auth(api, refresh_file=cred)
sub = Gen3Submission(api, auth)
query = Gen3Query(auth)
index = Gen3Index(auth)
file = Gen3File(auth)
metadata = Gen3Metadata(auth)
exp = Gen3Expansion(api,auth,sub) 
process = Gen3process( sub )
exp.get_project_ids()

Getting all project_ids you have access to in the data commons.
['MyFirstProgram-MyFirstProject']


['MyFirstProgram-MyFirstProject']

### Set up a temporary folder for downloading data files

In [3]:
vis_id = "prj161_vis_080223"
folder_name = f"tmp_download_{vis_id}"
current_directory = os.getcwd()
folder_path = os.path.join(current_directory, folder_name)

# Check if the directory already exists
if not os.path.exists(folder_path):
    # Create the directory and any necessary parent directories
    os.makedirs(folder_path)
    print(f"Directory '{folder_path}' created successfully.")
else:
    print(f"Directory '{folder_path}' already exists.")

Directory 'C:\Users\15177\OneDrive - Michigan State University\Documents\Projects\FAIRTox\DH_RealData\SRC_TZES004911_PRJ161\tmp_download_prj161_vis_080223' already exists.


---
<h1 align="center">Longitudinal body weight measurements</h1> 

Over the course of an experiment the study subjects will vary in body weight, sometimes diverging based on the treatment compounds. Here we will examine what effect repeated gavage with 2,3,7,8-Tetrachlorodibenzo-p-dioxin had on body weight over a 28 day treatment period.

---
## Build your subjects metadata table

We use the `TDCexpansion` python package to build a subject metadata table. In this example we are looking at a study in which male mice were treated with TCDD every 4 days for 28 days.

In [12]:
# Provide a list of projects you want to filter by
prog = 'MyFirstProgram'
proj = 'MyFirstProject'
StudyFilter = ['PRJ161']

# Get study details
study_df = process.build_study('MyFirstProgram', 'MyFirstProject', studies_to_match = StudyFilter)

# Build the subject table from TDC
subject_df = process.build_subject('MyFirstProgram', 'MyFirstProject', studies_to_match = StudyFilter)

### Print study details

In [13]:
print('Publications available at DOI: ' + study_df['DOI'].item().replace(":", "; ") + '\n')
print('This study was funded by: ' + study_df['support_source'].item() + ' ' + study_df['support_id'].item())

Publications available at DOI: 10.1093/toxsci/kfac109; 10.1093/nar/gkac019; 10.3390/receptors2020009; 10.1101/2022.10.05.510890

This study was funded by: NIEHS; Superfund Basic Research Program P42ES004911


### Collect bodyweight data measurements

In [15]:
# Build a dataframe to find the body weight data files for the study
weight_data = process.process_node_data('weight_measurement', ['subjects.submitter_id'], 
                                        program=prog, 
                                        project=proj) 

# Cleanup the weight data table
bw_data = weight_data[weight_data['submitter_id'] == 'PRJ161:BodyWeights'] # A more efficient method in development
bw_data = bw_data.dropna(axis=1) # Remove empty columns
bw_combinations = bw_data.drop_duplicates(subset=["file_name", "object_id"]) # Find only unique file names

# Collected linked filename and GUID to download correct data file
bw_filename = bw_combinations['file_name']
bw_objectid = bw_combinations['object_id']

### Cleanup metadata table and merge with bodyweight data

In [16]:
# Identify columns to be used for grouping and plotting
keep = ['subjects.submitter_id', 'sex', 'dose_amount', 'test_article_administration_zt', 
        'test_article_dtxsid', 'test_article_name', 'vehicle_dtxsid', 'vehicle_name', 'strain'
       ]

dataframes = [('subject_df', subject_df),('bw_data', bw_data)]
metadata = process.merge_dataframes(process.rename_overlapping(dataframes=dataframes))

# Remove the 'PRJ161:' string from the identifier to match the datafile input 
metadata["subjects.submitter_id"] = metadata["subjects.submitter_id"].str.replace("^PRJ161:", "", regex=True)
metadata = metadata[keep].drop_duplicates()
display(metadata.head())

,subjects.submitter_id,sex,dose_amount,test_article_administration_zt,test_article_dtxsid,test_article_name,vehicle_dtxsid,vehicle_name,strain
0,M37,male,0.1,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl
7,M38,male,0.1,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl
14,M39,male,0.1,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl
21,M55,male,0.0,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl
28,M56,male,0.0,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl


### Download the bodyweight data file(s)

To download the data files we use the [Gen3 data client](https://github.com/uc-cdis/cdis-data-client/releases/). This is done using the `GUID` which can be obtained from the code above or through the online portal.

In [18]:
print("Run the following commands using the gen3 client to download the data: \n")
print("cd " + folder_path + '\n')
profile = "fairtox" # You will have set this up when uploading the data files
file_identifier = str(bw_objectid.item())
cmd = r'gen3-client download-single --profile="{}" --guid="{}"'.format(profile, file_identifier)
print(cmd)

Run the following commands using the gen3 client to download the data: 

cd C:\Users\15177\OneDrive - Michigan State University\Documents\Projects\FAIRTox\DH_RealData\SRC_TZES004911_PRJ161\tmp_download_prj161_vis_080223

gen3-client download-single --profile="fairtox" --guid="dg.TDC/7e994d69-e5ce-4bb0-972d-1b3d87b1607c"


### Prepare the data for plotting

In [19]:
df = pd.read_table(folder_path + '/' + bw_filename.item())
df.head()

,subjects.submitter_id,days_relative_first_dose,measurement_ZT,value,is_terminal,comment
0,M01,0,01:00:00,17.2,False,NaN
1,M02,0,01:00:00,15.9,False,NaN
2,M03,0,01:00:00,16.1,False,NaN
3,M04,0,01:00:00,17.2,False,NaN
4,M05,0,01:00:00,15.7,False,NaN


In [20]:
merged_df = pd.merge(metadata, df, on='subjects.submitter_id')
merged_df.head()

,subjects.submitter_id,sex,dose_amount,test_article_administration_zt,test_article_dtxsid,test_article_name,vehicle_dtxsid,vehicle_name,strain,days_relative_first_dose,measurement_ZT,value,is_terminal,comment
0,M37,male,0.1,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl,0,01:00:00,13.2,False,NaN
1,M37,male,0.1,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl,1,01:00:00,14.2,False,NaN
2,M37,male,0.1,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl,2,01:00:00,14.7,False,NaN
3,M37,male,0.1,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl,3,01:00:00,15.5,False,NaN
4,M37,male,0.1,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl,4,01:00:00,16.3,False,NaN


---
## Plotly interactive bodyweight visualization

In [25]:
# Calculate the mean and standard error for each group
grouped_stats = merged_df.groupby(["dose_amount", "days_relative_first_dose", "sex"])["value"].agg(['mean', 'sem']).reset_index()

# Create the interactive plot using Plotly graph_objects
fig = go.Figure()

for (dose_amount, sex), group in grouped_stats.groupby(["dose_amount", "sex"]):
    # Find the mean value at days_relative_first_dose = 0 for each group
    mean_at_zero = group.loc[group["days_relative_first_dose"] == 0, "mean"].values[0]
    # Rescale y-values (mean) to start from the mean value at days_relative_first_dose = 0 for each group
    rescaled_mean = group["mean"] - mean_at_zero
    fig.add_trace(
        go.Scatter(
            x=group["days_relative_first_dose"],
            y=rescaled_mean,
            name=f"Dose: {dose_amount}, Sex: {sex.capitalize()}",
            mode="markers+lines",
            text=f"Dose: {dose_amount}, Sex: {sex.capitalize()}",
            error_y=dict(type='data', array=group['sem']),
        )
    )

# Set axis labels
# Set axis labels and layout
fig.update_layout(
    xaxis_title="Days Relative to First Dose",
    yaxis_title="Scaled average body weight",
    plot_bgcolor='white',  # Set background color to white
    paper_bgcolor='white',  # Set paper color to white
    showlegend=True,  # Show the legend
    legend=dict(
        bordercolor="black",  # Set legend border color
        borderwidth=1,  # Set legend border width
    ),
)

# Set the plot area border color
fig.update_xaxes(showline=True, linewidth=1, linecolor='black')  # Set x-axis border color
fig.update_yaxes(showline=True, linewidth=1, linecolor='black')  # Set y-axis border color

# Show the interactive plot
fig.show()


---
<h1 align="center">Terminal tissue weight measurements</h1> 

Here we explore how treatment affected the tissue weights both normalized to body weight and un-normalized.

---
## Build your sample metadata table

We use the `TDCexpansion` python package to build a sample metadata table. 

In [27]:
# Filter the samples by the subjects from the project obtained previously
SubjectFilter = subject_df['subjects.submitter_id'].unique().tolist()
sample_df = process.process_node_data('sample', ['subjects.submitter_id'], 'MyFirstProgram', 'MyFirstProject')

# Merge with the subjects to get dose/subject details
dataframes = [('subject_df', subject_df),('samples', sample_df)]
sample_df = process.merge_dataframes(process.rename_overlapping(dataframes=dataframes))

sample_df.head()

,subjects.type,subjects.project_id,subjects.submitter_id,euthanasia_date,euthanasia_method,euthanasia_zt,experiment_start_date,experiment_start_zt,subjects.provenance,sex,...,samples.submitter_id,biospecimen_anatomic_site,collection_protocol,date,method_of_sample_procurement,preservation_method,provenance,storage_vessel,volume,weight
0,subject,MyFirstProgram-MyFirstProject,PRJ161:M37,2019-10-20,Carbon dioxide asphyxiation,0,2019-09-22,0,NaN,male,...,PRJ161:L37_FFPE,Liver,A section of the left lateral lobe was excised...,2019-10-20,NaN,FFPE,"DataHarmonizer v1.4.10, Sample v4.3.1",Histology cassette,NaN,NaN
1,subject,MyFirstProgram-MyFirstProject,PRJ161:M37,2019-10-20,Carbon dioxide asphyxiation,0,2019-09-22,0,NaN,male,...,PRJ161:L37,Liver,Tissue was carefully excised and immediately f...,2019-10-20,NaN,Snap Frozen,"DataHarmonizer v1.4.10, Sample v4.3.1",2 mL screwcap tube,NaN,NaN
2,subject,MyFirstProgram-MyFirstProject,PRJ161:M37,2019-10-20,Carbon dioxide asphyxiation,0,2019-09-22,0,NaN,male,...,PRJ161:KR37,Kidney,Kidneys were excised during necrospy and colle...,2019-10-27,NaN,Snap Frozen,"DataHarmonizer v1.4.10, Sample v4.3.1",2 mL screwcap tube,NaN,NaN
3,subject,MyFirstProgram-MyFirstProject,PRJ161:M37,2019-10-20,Carbon dioxide asphyxiation,0,2019-09-22,0,NaN,male,...,PRJ161:S37,Blood,Blood was collected via cardiac puncture and a...,2019-10-20,NaN,Snap Frozen,"DataHarmonizer v1.4.10, Sample v4.3.1",0.5 mL microtube,NaN,NaN
4,subject,MyFirstProgram-MyFirstProject,PRJ161:M37,2019-10-20,Carbon dioxide asphyxiation,0,2019-09-22,0,NaN,male,...,PRJ161:L37_FFPE,Liver,A section of the left lateral lobe was excised...,2019-10-20,NaN,FFPE,"DataHarmonizer v1.4.10, Sample v4.3.1",Histology cassette,NaN,NaN


In [28]:
# Sample weight (sw) data
sw_data = weight_data[weight_data['submitter_id'] == 'PRJ161:SampleWeights'] # A more efficient method in development
sw_data = sw_data.dropna(axis=1)
sw_combinations = sw_data.drop_duplicates(subset=["file_name", "object_id"])
sw_filename = sw_combinations['file_name']
sw_objectid = sw_combinations['object_id']

### Download the sample weight data file

In [29]:
print("Run the following commands using the gen3 client to download the data: \n")
print("cd " + folder_path + '\n')
profile = "fairtox"
file_identifier = str(sw_objectid.item())
cmd = r'gen3-client download-single --profile="{}" --guid="{}"'.format(profile, file_identifier)
print(cmd)

Run the following commands using the gen3 client to download the data: 

cd C:\Users\15177\OneDrive - Michigan State University\Documents\Projects\FAIRTox\DH_RealData\SRC_TZES004911_PRJ161\tmp_download_prj161_vis_080223

gen3-client download-single --profile="fairtox" --guid="dg.TDC/2b95a690-8eda-4180-ab99-2f24870a2dcc"


### Create metadata table and merge

In [38]:
keep = ['subjects.submitter_id', 'samples.submitter_id', 'sex', 'dose_amount', 'test_article_administration_zt', 
        'test_article_dtxsid', 'test_article_name', 'vehicle_dtxsid', 'vehicle_name', 'strain', 'biospecimen_anatomic_site'
       ]

# Remove the 'PRJ161:' string from the identifier to match the datafile input 
sample_df["subjects.submitter_id"] = sample_df["subjects.submitter_id"].str.replace("^PRJ161:", "", regex=True)
sample_df["samples.submitter_id"] = sample_df["samples.submitter_id"].str.replace("^PRJ161:", "", regex=True)
sample_df = sample_df[keep].drop_duplicates()
display(sample_df.head())

,subjects.submitter_id,samples.submitter_id,sex,dose_amount,test_article_administration_zt,test_article_dtxsid,test_article_name,vehicle_dtxsid,vehicle_name,strain,biospecimen_anatomic_site
0,M37,L37_FFPE,male,0.1,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl,Liver
1,M37,L37,male,0.1,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl,Liver
2,M37,KR37,male,0.1,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl,Kidney
3,M37,S37,male,0.1,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl,Blood
28,M38,S38,male,0.1,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl,Blood


### Prepare the data for plotting

In [39]:
df = pd.read_table(folder_path + '/' + sw_filename.item())
df.head()

,samples.submitter_id,days_relative_first_dose,measurement_ZT,value,is_terminal,comment
0,L01,28,0:00:00,1.2128,True,NaN
1,L02,28,0:00:00,1.2397,True,NaN
2,L03,28,0:00:00,1.3836,True,NaN
3,L04,28,0:00:00,1.4130,True,NaN
4,L05,28,0:00:00,1.4509,True,NaN


In [40]:
bw_df = merged_df[merged_df['days_relative_first_dose'].isin(df['days_relative_first_dose'].unique().tolist())]

input_data = pd.merge(sample_df, bw_df, on='subjects.submitter_id')
input_data = pd.merge(input_data, df, on='samples.submitter_id')

input_data = input_data[input_data['is_terminal_x'] == True]
input_data.head()

,subjects.submitter_id,samples.submitter_id,sex_x,dose_amount_x,test_article_administration_zt_x,test_article_dtxsid_x,test_article_name_x,vehicle_dtxsid_x,vehicle_name_x,strain_x,...,days_relative_first_dose_x,measurement_ZT_x,value_x,is_terminal_x,comment_x,days_relative_first_dose_y,measurement_ZT_y,value_y,is_terminal_y,comment_y
0,M37,L37,male,0.1,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl,...,28,01:00:00,21.0,True,NaN,28,0:00:00,1.3268,True,NaN
1,M38,L38,male,0.1,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl,...,28,01:00:00,21.3,True,NaN,28,0:00:00,1.2230,True,NaN
2,M39,L39,male,0.1,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl,...,28,01:00:00,21.2,True,NaN,28,0:00:00,1.2504,True,NaN
3,M55,L55,male,0.0,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl,...,28,01:00:00,18.9,True,NaN,28,0:00:00,1.0466,True,NaN
4,M56,L56,male,0.0,0,DTXSID2021315,TCDD,DTXSID9033971,Sesame Oil,C57BL/6NCrl,...,28,01:00:00,19.7,True,NaN,28,0:00:00,1.1512,True,NaN


In [41]:
input_data['relative_weight'] = (input_data['value_y']/input_data['value_x'])

# Convert "dose_amount" to a categorical variable with a specific order
dose_order = sorted(input_data["dose_amount_x"].unique())
input_data["dose_amount_x"] = pd.Categorical(input_data["dose_amount_x"], categories=dose_order, ordered=True)

# Sort the DataFrame based on "dose_amount"
input_data.sort_values("dose_amount_x", inplace=True)

### Plotly interactive visualization of normalized liver weights
---
### Histogram of relative liver weights

In [49]:
from plotly.subplots import make_subplots  # Add this line to import make_subplots

df = input_data

# Create subplots for each biospecimen_anatomic_site
fig = make_subplots(rows=1, cols=len(df["biospecimen_anatomic_site"].unique()), shared_yaxes=True)

for i, site in enumerate(df["biospecimen_anatomic_site"].unique(), 1):
    # Filter the data for the current biospecimen_anatomic_site
    site_data = df[df["biospecimen_anatomic_site"] == site]

    # Create a histogram trace for each sex and dose combination
    for sex in site_data["sex_x"].unique():
        for dose in site_data["dose_amount_x"].unique():
            trace_data = site_data[(site_data["sex_x"] == sex) & (site_data["dose_amount_x"] == dose)]
            trace = go.Histogram(x=trace_data["relative_weight"], nbinsx=8, name=f"{sex} - Dose {dose}")
            fig.add_trace(trace, row=1, col=i)

# Update layout and axis labels
fig.update_layout(
    title_text="Histogram by Sex and Dose",
    xaxis_title="Relative Weight",
    yaxis_title="Count",
    plot_bgcolor='white',  # Set background color to white
    paper_bgcolor='white',  # Set paper color to white
    showlegend=True,  # Hide the legend for individual histograms
)

# Set the plot area border color
fig.update_xaxes(showline=True, linewidth=1, linecolor='black')  # Set x-axis border color
fig.update_yaxes(showline=True, linewidth=1, linecolor='black')  # Set y-axis border color

# Show the plot
fig.show()

### Boxplot of relative liver weights for each group

In [54]:
df = input_data

# Convert "dose_amount_x" to a categorical variable with a specific order (if needed)
dose_order = sorted(df["dose_amount_x"].unique())
df["dose_amount_x"] = pd.Categorical(df["dose_amount_x"], categories=dose_order, ordered=True)

# Function to handle conversion of numeric and float values to string
def format_dose_value(value):
    if isinstance(value, (int, float)):
        return str(value).rstrip('0').rstrip('.')
    return value

# Convert "dose_amount_x" to a string and handle the conversion of numeric and float values
df["dose_amount_x"] = df["dose_amount_x"].apply(format_dose_value)

# Create a box plot using Plotly graph objects
box_fig = go.Figure()
for site in df["biospecimen_anatomic_site"].unique():
    box_data = df[df["biospecimen_anatomic_site"] == site]
    box_fig.add_trace(
        go.Box(
            x=box_data["dose_amount_x"],
            y=box_data["relative_weight"],
            name=site,
            boxpoints=False,  # Do not show points in the box plot
        )
    )

# Add jitter points to the same figure
for site in df["biospecimen_anatomic_site"].unique():
    jitter_data = df[df["biospecimen_anatomic_site"] == site]
    box_fig.add_trace(
        go.Scatter(
            x=jitter_data["dose_amount_x"],
            y=jitter_data["relative_weight"],
            name=f"{site} Jitter",
            mode="markers",
            marker=dict(size=8),
            showlegend=False,  # To avoid duplicating the legend
        )
    )

box_fig.update_layout(
    xaxis_title="Dose Amount",
    yaxis_title="Relative Weight",
    title="Box Plot with Jitter Points by Sex and Tissue",
    plot_bgcolor='white',  # Set background color to white
    paper_bgcolor='white',  # Set paper color to white
)

# Set the plot area border color
box_fig.update_xaxes(showline=True, linewidth=1, linecolor='black')  # Set x-axis border color
box_fig.update_yaxes(showline=True, linewidth=1, linecolor='black')  # Set y-axis border color

# Show the plot
box_fig.show()

### Plotly interactive visualization of un-normalized liver weights
---
### Boxplot of liver weights

In [56]:
df = input_data

# Convert "dose_amount_x" to a categorical variable with a specific order (if needed)
dose_order = sorted(df["dose_amount_x"].unique())
df["dose_amount_x"] = pd.Categorical(df["dose_amount_x"], categories=dose_order, ordered=True)

# Create a box plot using Plotly graph objects
box_fig = go.Figure()
for site in df["biospecimen_anatomic_site"].unique():
    box_data = df[df["biospecimen_anatomic_site"] == site]
    box_fig.add_trace(
        go.Box(
            x=box_data["dose_amount_x"],
            y=box_data["value_y"],
            name=site,
            boxpoints=False,  # Do not show points in the box plot
        )
    )

# Add jitter points to the same figure
for site in df["biospecimen_anatomic_site"].unique():
    jitter_data = df[df["biospecimen_anatomic_site"] == site]
    box_fig.add_trace(
        go.Scatter(
            x=jitter_data["dose_amount_x"],
            y=jitter_data["value_y"],
            name=f"{site} Jitter",
            mode="markers",
            marker=dict(size=8),
            showlegend=False,  # To avoid duplicating the legend
        )
    )

box_fig.update_layout(
    xaxis_title="Dose Amount",
    yaxis_title="Relative Weight",
    title="Box Plot with Jitter Points by Sex and Tissue",
    plot_bgcolor='white',  # Set background color to white
    paper_bgcolor='white',  # Set paper color to white
)

# Set the plot area border color
box_fig.update_xaxes(showline=True, linewidth=1, linecolor='black')  # Set x-axis border color
box_fig.update_yaxes(showline=True, linewidth=1, linecolor='black')  # Set y-axis border color

# Show the plot
box_fig.show()